In [1]:
import pandas as pd
import numpy as np

## Read csv files

In [3]:
books = pd.read_csv('BX-Books.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

/tmp/ipykernel_19073/800602712.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep=';', on_bad_lines='skip', encoding='latin-1')


In [4]:
print(books.shape)
books.head()

(271360, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

In [6]:
print(ratings.shape)
ratings.head()

(1149780, 3)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
users = pd.read_csv('BX-Users.csv', sep=';', on_bad_lines='skip', encoding='latin-1')

In [8]:
print(users.shape)
users.head()

(278858, 3)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [9]:
users_ratings = users.merge(ratings, on='User-ID')

In [10]:
df = users_ratings.merge(books, on='ISBN')

In [11]:
df.head(1)

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2,"stockton, california, usa",18.0,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...


## Feature engineering

In [13]:
df = df[['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']].copy()

In [14]:
df.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating', 'Book-Title': 'title', 'Image-URL-L': 'image_url'}, inplace=True)

In [15]:
df.isnull().sum()

user_id                0
ISBN                   0
rating                 0
title                  0
Book-Author            2
Year-Of-Publication    0
Publisher              2
image_url              4
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
df.duplicated().sum()

0

In [18]:
y = df['user_id'].value_counts()>20

In [19]:
y[y].shape

(6374,)

In [20]:
df = df[df.user_id.isin(y[y].index)]

## take only those books which got ratings more than 10, to make the matrix small

In [22]:
number_rating = df.groupby('title')['rating'].count().reset_index()

In [23]:
number_rating.rename(columns={'rating':'num_of_rating'}, inplace=True)

In [24]:
final_df = df.merge(number_rating, on='title')

In [25]:
# Lets take those books which got at least 10 rating of user

final_df = final_df[final_df['num_of_rating'] >= 10]

In [26]:
final_df.shape

(398482, 9)

In [27]:
final_df.isnull().sum()

user_id                0
ISBN                   0
rating                 0
title                  0
Book-Author            0
Year-Of-Publication    0
Publisher              0
image_url              0
num_of_rating          0
dtype: int64

In [28]:
final_df.duplicated().sum()

0

In [29]:
final_df.head()

,user_id,ISBN,rating,title,Book-Author,Year-Of-Publication,Publisher,image_url,num_of_rating
0,242,0553278223,10,The Martian Chronicles,RAY BRADBURY,1984,Spectra,http://images.amazon.com/images/P/0553278223.0...,51
1,242,0971880107,0,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,1108
5,242,3257208626,8,Fahrenheit 451,Ray Bradbury,1994,Distribooks Inc,http://images.amazon.com/images/P/3257208626.0...,271
21,242,3498020862,8,Die Korrekturen.,Jonathan Franzen,2002,"Rowohlt, Reinbek",http://images.amazon.com/images/P/3498020862.0...,12
28,243,0060915544,10,The Bean Trees,Barbara Kingsolver,1989,Perennial,http://images.amazon.com/images/P/0060915544.0...,258


In [30]:
# Lets create a pivot table
pivot_table = final_df.pivot_table(columns='user_id', index='title', values= 'rating')

In [31]:
pivot_table

user_id,242,243,254,383,388,408,446,487,503,507,...,278194,278202,278221,278356,278418,278535,278582,278633,278843,278851
title,,,,,,,,,,,,,,,,,,,,,
Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Q-Space (Star Trek The Next Generation, Book 47)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Q-Zone (Star Trek The Next Generation, Book 48)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\O\"" Is for Outlaw""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
pivot_table.fillna(0, inplace=True)

In [33]:
pivot_table

user_id,242,243,254,383,388,408,446,487,503,507,...,278194,278202,278221,278356,278418,278535,278582,278633,278843,278851
title,,,,,,,,,,,,,,,,,,,,,
Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Space (Star Trek The Next Generation, Book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Zone (Star Trek The Next Generation, Book 48)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# model training

In [35]:
from scipy.sparse import csr_matrix

In [36]:
sparse_matrix = csr_matrix(pivot_table)

In [37]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [38]:
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute')

Example to check if it works

In [39]:
# Nearest neighbors gives two outputs - distance and indices
distance, indices = model.kneighbors(pivot_table.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [76]:
distance

array([[ 0.        , 12.20655562, 13.96424004, 15.03329638, 15.03329638,
        15.03329638]])

In [78]:
indices

array([[  237, 11836,  1748,    10,    19,   198]])

In [82]:
for i in range(len(indices)):
    print(pivot_table.index[indices[i]])

Index(['A Light in the Window (The Mitford Trilogy : Book 2)',
       'The Stormy Petrel', 'California Fire and Life',
       '101 Hopelessly Hilarious Jokes', '1ST AMONG EQUALS',
       'A Habit of the Blood'],
      dtype='object', name='title')


In [86]:
#keeping books name
book_names = pivot_table.index

In [93]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_df,open('artifacts/final_rating.pkl','wb'))
pickle.dump(pivot_table,open('artifacts/book_pivot.pkl','wb'))

## Testing model

In [134]:
def recommend_book(book_name):
    book_id = np.where(pivot_table.index == book_name)[0][0]
    distance, indices = model.kneighbors(pivot_table.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(indices)):
            books = pivot_table.index[indices[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggested books are: \n")
                else:
                    print(j)

In [136]:
recommend_book('Zoya')

You searched 'Zoya'

The suggested books are: 

Blood Relations
Perfect Stranger
Everything to Gain
Half Moon Street (Charlotte &amp; Thomas Pitt Novels (Paperback))
Cooks Overboard (Angie Amalfi Mysteries)
